In [26]:
# import packages
from tensorflow.keras.layers import BatchNormalization, Conv2D, AveragePooling2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import Conv1D, AveragePooling1D, MaxPooling1D, ZeroPadding1D
from tensorflow.keras.layers import Activation, Dense,  Flatten, Input, add
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np

#define resnet class
class ResNet:
  @staticmethod
  #create one residual block
  def residual_module(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
    shortcut = data
    
    bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
    act1 = Activation("relu")(bn1)
    conv1 = Conv1D(int(K*0.25), 1, use_bias=False, kernel_regularizer=l2(reg))(act1)

    bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
    act2 = Activation("relu")(bn2)
    conv2 = Conv1D(int(K * 0.25), 1, strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act2)
    
    bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
    act3 = Activation("relu")(bn3)
    conv3 = Conv1D(K, 1, use_bias=False, kernel_regularizer=l2(reg))(act3)

    print(bn2.shape, act2.shape, conv2.shape, bn3.shape)
    print(shortcut.shape)

    if red:
      shortcut = Conv1D(K, 1, strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)

    x = add([conv3, shortcut])

    return x

  @staticmethod
  #build a model out of resblocks
  def build(inputShape, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
    chanDim = -1
    
    inputs = Input(shape=inputShape)
    x = BatchNormalization(axis=chanDim, epsilon = bnEps, momentum=bnMom)(inputs)

    for i in range(0, len(stages)):
      if i == 0:
        stride = 1
      else:
        stride = 2
      
      x = ResNet.residual_module(x, filters[i + 1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

      for j in range(0, stages[i] - 1):
        x = ResNet.residual_module(x, filters[i + 1], 1, chanDim, bnEps=bnEps, bnMom=bnMom)
      
      x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
      x = Activation("relu")(x)
      x = AveragePooling1D(8)(x)

      x = Flatten()(x)
      x = Dense(classes, kernel_regularizer=l2(reg))(x)
      x = Activation("softmax")(x)

      model = Model(inputs, x, name="resnet")

      return model

In [ ]:
#some hyperparameters
batch_size = 32
epochs = 10
verbose = 2

#load data (specific to google drive)
x_train = np.load('/content/drive/My Drive/ML Group/Datasets/data/X_reference.npy')
y_train = np.load('/content/drive/My Drive/ML Group/Datasets/data/y_reference.npy')
x_test = np.load('/content/drive/My Drive/ML Group/Datasets/data/X_test.npy')
y_test = np.load('/content/drive/My Drive/ML Group/Datasets/data/y_test.npy')

#add depth channel
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

#input shape is everything except for the number of samples 
#number of classes is the number of unique items in y
in_shape = x_train.shape[1:]
n_classes = len(np.unique(y_train))

#for debugging
print(x_train.shape, in_shape)

#build model
model = ResNet.build(inputShape=in_shape, classes=n_classes, stages=(3, 4, 6), filters=(64, 128, 256, 512))

#compile, summarize, train model
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()
model.fit(x_train, y_train, batch_size=batch_size,epochs=epochs, verbose=verbose)

#test accuracy
accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=verbose)
print(accuracy)

#delete model
del model
